In [65]:
import openai
import pandas as pd
import numpy as np
import string
import json
import transformers

In [66]:
model2="TheBloke/Llama-2-7B-Chat-GGUF/llama-2-7b-chat.Q5_K_M.gguf"
model3="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf" #json ok frase ok (11seg)
model17="failspy/kappa-3-phi-3-4k-instruct-abliterated-GGUF/ggml-model-f16.gguf" #bueno como llama3


In [67]:
#user_query = "mi tarjeta es gratis pero la de mi mujer no"
user_query = "¿Cómo activo el fraccionamiento automático?"

In [68]:
from sentence_transformers import SentenceTransformer

def embeddings_GIST(docs: list[str]):
    model = SentenceTransformer("avsolatorio/GIST-small-Embedding-v0")
    embeddings = model.encode(docs, convert_to_tensor=False)
    return embeddings

In [69]:
# Defining cosine similarity 
def compute_cosine_similarity(a,b): 
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return (cos_sim)

In [70]:
df_embed_uoc = pd.read_pickle('data/variaciones_all_comp_embedding_uoc_20240427.pkl')
df_embed_uoc.head()

,CODIGO_FAQ,embeddings_mini,embeddings_bert,embeddings_spacy,embeddings_mxbai,embeddings_GIST,motivo
0,433243,"[-0.049297784, 0.06306262, -0.052778855, -0.04...","[-0.049297784, 0.06306262, -0.052778855, -0.04...","[0.06396655, -1.0102829, 0.100738235, 1.127394...","[0.02481748, -0.03403738, -0.14437212, 0.72762...","[-0.030527227, 0.0026669737, 0.01400228, 0.012...",NaN
1,419915,"[-0.026050719, -0.07427596, -0.05052995, -0.07...","[-0.026050719, -0.07427596, -0.05052995, -0.07...","[0.84611166, 0.28720096, 0.834923, -1.2378342,...","[0.48073822, -0.018162876, 0.223723, 0.5368666...","[-0.043238733, -0.026242362, 0.0061765215, -0....",NaN
2,421998,"[-0.12729552, 0.06905968, -0.080994666, -0.075...","[-0.12729552, 0.06905968, -0.080994666, -0.075...","[0.35114053, -0.24000733, 0.6779311, -0.876988...","[-0.028965719, 0.4119485, 0.04523738, 0.877965...","[-0.0527733, 0.0033994697, 0.02993245, -0.0139...",NaN
3,423029,"[-0.1008265, 0.026075443, -0.042023044, -0.106...","[-0.1008265, 0.026075443, -0.042023044, -0.106...","[-0.12797742, -0.97483176, 0.7096605, 0.197163...","[0.49374753, -0.26194438, 0.089569956, 0.30814...","[-0.031788237, 0.008151135, 0.04830003, 0.0147...",NaN
4,422618,"[-0.06829783, 0.0648448, -0.07408309, -0.08305...","[-0.06829783, 0.0648448, -0.07408309, -0.08305...","[0.054455537, -0.6434433, 0.06104442, 0.659452...","[0.031954594, -0.016352661, -0.09663524, 0.226...","[-0.03212525, -0.019531578, 0.03505223, 0.0109...",NaN


In [71]:
e = embeddings_GIST(user_query)
print (e)
df_embed_uoc['score_GIST'] = df_embed_uoc['embeddings_GIST'].apply(lambda x: compute_cosine_similarity(x, e))

[-2.23927740e-02 -2.75972709e-02  3.62064876e-02 -1.63107533e-02
 -2.92220560e-04  2.45558806e-02  2.24027652e-02  3.04355752e-02
 -3.09610330e-02 -1.71935074e-02 -1.48985730e-02 -8.97379033e-03
  1.40590314e-02  1.38739152e-02  3.95910777e-02 -1.05411047e-02
  4.70518228e-03  2.86190379e-02  6.41230075e-03  4.23726961e-02
  5.84802553e-02  3.81777459e-03 -5.08837663e-02 -2.85346843e-02
 -9.76181123e-03  4.13180999e-02 -8.06099270e-03 -2.66627781e-02
 -5.69040254e-02 -1.98928267e-01  4.99768090e-03 -8.51940736e-02
  8.26102644e-02 -2.80836429e-02 -1.43505568e-02 -2.98993252e-02
 -9.80595872e-02  1.13886688e-02 -4.14335877e-02  2.54194112e-03
 -8.78440565e-04  1.04382811e-02 -7.28127873e-03 -5.02270870e-02
 -1.01010231e-02 -3.81094106e-02  2.73234788e-02  1.76800527e-02
 -1.37345186e-02  9.92874242e-03 -3.81964408e-02  6.03615120e-02
  3.84227745e-02  4.96573932e-03 -4.28577513e-02  2.67829485e-02
  3.83092538e-02  3.56287248e-02  1.04763247e-01  4.66977544e-02
  7.00729787e-02  4.82363

In [72]:
# Get index of the 5 highest score_GIST's rows 
indices_max_scores_GIST= df_embed_uoc['score_GIST'].nlargest(1).index

# print scores and codes
codes_max_scores_GIST = df_embed_uoc.loc[indices_max_scores_GIST, ['CODIGO_FAQ','score_GIST']] 
print(codes_max_scores_GIST)

    CODIGO_FAQ  score_GIST
29      422999     0.86187


In [73]:
df_corpus_uoc = pd.read_pickle('data/corpus_con_variaciones_20240427_uoc_ofuscado.pkl')
df_corpus_uoc.head()
df_corpus_docs = df_corpus_uoc[['CODIGO_FAQ','title_found','body texto']]
df_corpus_docs.head()

,CODIGO_FAQ,title_found,body texto
0,433243,Visa Lidl para empleados del grupo Lidl,¿A quién está dirigida? La Visa Lidl Empleado ...
1,419915,¿Cómo puedo anular el cobro de una liquidación...,¿Cómo puedo anular el cobro de una liquidación...
2,421998,¿Cómo puedo modificar un contrato de tarjeta p...,¿Cómo puedo modificar un contrato de tarjeta p...
3,423029,¿Cómo modificar la forma o importe de cobro de...,¿Cuál es la comisión por recargar una tarjeta ...
4,422618,¿Cuáles son los plazos de recepción de las tar...,¿De que depende el plazo de recepción de las t...


In [74]:
def get_documents_by_code(codes, df_docs):
    return df_corpus_docs[df_docs['CODIGO_FAQ'].isin(codes)] 

In [75]:
def get_top_documents(codes, df_docs):
    added_docs = 0
    docs = ''
    document = get_documents_by_code(codes, df_docs)
    print (document)
    
    for idx, doc in document.iterrows():
        docs += "#Documento " + str(doc['CODIGO_FAQ']) + ": "  + doc['title_found'] + " - " + doc['body texto'] + "\n"
        added_docs += 1
    return docs

In [76]:
prompt_docs = get_top_documents(codes_max_scores_GIST['CODIGO_FAQ'].tolist(),df_corpus_docs)
print(prompt_docs)

    CODIGO_FAQ                                        title_found  \
29      422999  ¿Cómo gestionar el perfil de fraccionamiento a...   

                                           body texto  
29  A través del perfil de fraccionamiento se gest...  
#Documento 422999: ¿Cómo gestionar el perfil de fraccionamiento automático de una tarjeta? - A través del perfil de fraccionamiento se gestionan las condiciones que han de cumplir las operaciones para poder fraccionarse de forma automática. 
Alta del perfil
expandir
Al dar de alta el perfil de fraccionamiento automático se debe indicar el importe a partir del cual se fraccionarán las operaciones y qué modalidad de fraccionamiento se desea. A partir de ese momento, todas las operaciones que cumplan con los requisitos informados, se fraccionarán automáticamente. 
En la modalidad de fraccionamiento con cobro de comisión, el tramo va de 210€ a 500€ por lo tanto, no se fraccionará automáticamente, las operaciones de importe superior a 500€.
Se 

In [77]:
prompt2 = "You are an assistant that responds to queries from UOCBank employees about internal documentation. The queries you receive are doubts that an employee has because a customer has come to the office and has a question that the employee cannot answer.\nEmployee query: {user_query}\nYour goal is to answer the previous question clearly and concisely following the following instructions:\n- The output format must ALWAYS be a single json with two fields, the \"response\" field and the \"sources\" field, without any additional text.\n- The \"response\" field must contain the necessary information to answer the question and the \"sources\" field a list of the documents from which the answer has been obtained.\n- All the information in the generated response must come ONLY from the provided documents marked between triple #.\n- Use the meaning and the entire context of the employee's query. If you are not completely sure that the answer is in the documents, you must indicate it in the \"response\" field and leave the sources field empty.\n- If you consider that you do not have enough specific information in the documents, say that you have not been able to answer in the \"response\" field and leave the sources field as an empty list: \"[]\".\n- Do not assume anything that is not explicit in the query or in the documents. It is critical that you do not make anything up and only respond with the information in the documents.\n- Never invent answers or information that is not used in the documents. You must use the information from each document in its context, without taking any phrase out of context.\n- Do not add the origin of the information in the \"response\" field, that is, never add references like \"More information in document XXXXX\" or \"You can consult document XXXXXX\".\n- The sources field must consist of the identifiers of the documents used for the answer in list format, e.g. [\"817231\", \"444444\"]. You only have to add to the list the documents whose content you have used for the answer. If you do not use the content of a document for the answer, you must not add it to the list.\n- If you have found sources, you must order them from most to least important in the response.\n- The response can have a maximum of 100 words. If the response is too long, summarize it to provide the information.\n- The \"response\" field must include HTML markers <b> and <\\b> to make the text bold later. It can also include markers like <BR>, <li>, etc. to make the output more visible.\nThe documents to generate the response are the following:\n###Documents:\n{documents}\nThe output format must ALWAYS be as follows:\n{\n    \"response\": \"Answer\",\n    \"sources\": [\"123456\", \"658194\", ...]\n}. The answer MUST BE ALWAYS in SPANISH."

In [78]:
final_prompt = prompt2.replace("{user_query}", user_query)

In [79]:
prompt_data = final_prompt.replace("{documents}", prompt_docs)

In [80]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

print ("Prompt: ", prompt_data)
print("*****CALLING MODEL 2*****")
completion = client.chat.completions.create(
  model=model3,
  messages = [{ "role": "system", "content": prompt_data }],
  temperature=0.0,
  #max_tokens=100,
  #stream=True,
)
'''
for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
'''
print("Result:",completion)
print(completion.choices[0].message.content)

Prompt:  You are an assistant that responds to queries from UOCBank employees about internal documentation. The queries you receive are doubts that an employee has because a customer has come to the office and has a question that the employee cannot answer.
Employee query: ¿Cómo activo el fraccionamiento automático?
Your goal is to answer the previous question clearly and concisely following the following instructions:
- The output format must ALWAYS be a single json with two fields, the "response" field and the "sources" field, without any additional text.
- The "response" field must contain the necessary information to answer the question and the "sources" field a list of the documents from which the answer has been obtained.
- All the information in the generated response must come ONLY from the provided documents marked between triple #.
- Use the meaning and the entire context of the employee's query. If you are not completely sure that the answer is in the documents, you must ind

Result: ChatCompletion(id='chatcmpl-43v8wczr6jnkldts34mw1h', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='system<|end_header_id|>\n{:output}\n{\n    "response": "<b>Cómo activo el fraccionamiento automático?</b> Para dar de alta el perfil de fraccionamiento automático, debes seleccionar la tarjeta que deseas y indicar el importe a partir del cual se fraccionará y qué modalidad de fraccionamiento se desea. A partir de ese momento, todas las operaciones que cumplan con los requisitos informados, se fraccionarán automáticamente.",\n    "sources": ["422999"]\n}', role='assistant', function_call=None, tool_calls=None))], created=1717256495, model='lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=110, prompt_tokens=111, total_tokens=221))
system<|end_header_id|>
{:output}
{
    "response": "<b>Cómo activo el 